In [1]:
from preprocessing import Audio
import tensorflow as tf
import tf_train as tft
import numpy as np
import os
import matplotlib.pyplot as plt
import scipy as sp
import librosa as li
import librosa.display
import IPython.display as ipd

%matplotlib inline
#tf.debugging.set_log_device_placement(True)
GPU = tf.config.list_physical_devices('GPU')
CPU = tf.config.list_physical_devices('CPU')
DEVICE = GPU[0].name if GPU else CPU[0].name
DEVICE

'/physical_device:GPU:0'

<h1>Подготовка данных для обучения и тестирования</h1>

In [2]:
def get_data():
    def norm(x):
        std = np.std(x, ddof=1)
        return (x - np.mean(x)) / std

    PATH_0 = r'AudioData/0 Данные' # negative class
    PATH_1 = r'AudioData/0 Шум' # noise
    PATH_2 = r'AudioData/1 Лада' # positive class

    PATHS = li.util.find_files(PATH_0)[:1700] + li.util.find_files(PATH_1) + li.util.find_files(PATH_2)
    N = len(PATHS)

    np.random.shuffle(PATHS)

    DATA = []
    LABELS = []
    for i, path in enumerate(PATHS):
        sound = Audio(path)
        e_parts = sound.augmented()
        for e in e_parts: 
            DATA.append(norm(e))
            LABELS.append(sound.label)
        print('Loading {:.1f}%'.format(i / N * 100))
        ipd.clear_output(wait=True)

    return (np.array(DATA), np.array(LABELS))

In [3]:
data, labels = get_data()

ValueError: operands could not be broadcast together with shapes (14225,) (28450,) 

In [52]:
data.shape, labels.shape

((2432, 20), (2432,))

In [53]:
BATCH_SIZE = len(data)
test_data = np.reshape(data, [BATCH_SIZE, 1, 20])
test_labels = np.reshape(labels, [1, BATCH_SIZE])

dataset = tf.data.Dataset.from_tensor_slices((test_data, labels))
dataset = dataset.batch(BATCH_SIZE)
dataset

<BatchDataset shapes: ((None, 1, 20), (None,)), types: (tf.float64, tf.int32)>

<h1>Создание модели нейронной сети</h1>

In [54]:
model = tf.keras.Sequential([
    tf.keras.layers.InputLayer(input_shape=(1, 20,)),
    tf.keras.layers.GRU(20, kernel_regularizer=tf.keras.regularizers.l2(0.01), reset_after=False),
    tf.keras.layers.Dense(2, activation='softmax')
], name='Lada_GRU-v2')

model.summary()

Model: "Lada_GRU-v2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru_5 (GRU)                  (None, 20)                2460      
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 42        
Total params: 2,502
Trainable params: 2,502
Non-trainable params: 0
_________________________________________________________________


<h1>Обучение</h1>

In [55]:
tft.train(
    train_dataset=dataset,
    model=model,
    epochs=5000,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[
        tft.metrics.Recall(),
        tft.metrics.Precision(),
        tft.metrics.AUC(),
    ],
    frequency=100
)

Epoch 0 -- loss_value: 0.741123, Recall: 0.583333, Precision: 0.217805, AUC: 0.565417, time - 0.060s
Epoch 100 -- loss_value: 0.210798, Recall: 0.700793, Precision: 0.660296, AUC: 0.811458, time - 3.621s
Epoch 200 -- loss_value: 0.167081, Recall: 0.722372, Precision: 0.731743, AUC: 0.832585, time - 4.773s
Epoch 300 -- loss_value: 0.134294, Recall: 0.752638, Precision: 0.772303, AUC: 0.852354, time - 5.415s
Epoch 400 -- loss_value: 0.107094, Recall: 0.780780, Precision: 0.800336, AUC: 0.869353, time - 5.690s
Epoch 500 -- loss_value: 0.086873, Recall: 0.803495, Precision: 0.823528, AUC: 0.883152, time - 5.281s
Epoch 600 -- loss_value: 0.071854, Recall: 0.825118, Precision: 0.843068, AUC: 0.895971, time - 5.383s
Epoch 700 -- loss_value: 0.059976, Recall: 0.845039, Precision: 0.859431, AUC: 0.907593, time - 4.929s
Epoch 800 -- loss_value: 0.050281, Recall: 0.862429, Precision: 0.873276, AUC: 0.917698, time - 4.915s
Epoch 900 -- loss_value: 0.042609, Recall: 0.876937, Precision: 0.884630, A

In [56]:
model.save(r'models\v2\{}.h5'.format(model.name))

In [8]:
model = tf.keras.models.load_model(r'models\Mira-v1.0.h5')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                252       
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 26        
Total params: 278
Trainable params: 278
Non-trainable params: 0
_________________________________________________________________


In [37]:
with tf.device("CPU"):
    predictions = tft.predict_classes(model=model, features=test_data)

In [38]:
n = 0

for i in range(len(labels)):
    n += int(labels[i] == predictions[i])

print(n, n / len(labels) * 100)

3361 100.0


In [41]:
predictions[60:70], labels[60:70]

(<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 1, 1, 0, 0, 0, 0, 3, 0], dtype=int64)>,
 array([0, 0, 1, 1, 0, 0, 0, 0, 3, 0]))

In [40]:
predictions[:15], labels[:15]

(<tf.Tensor: shape=(15,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], dtype=int64)>,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]))

In [30]:
labels[60:70]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [69]:
filename = 'test.tfrecord'
writer_data = tf.data.experimental.TFRecordWriter("datasets\Mira\data.tfrecord")
writer_labels = tf.data.experimental.TFRecordWriter("datasets\Mira\labels.tfrecord")
test_data = tf.data.Dataset.from_tensor_slices(data)
test_labels = tf.data.Dataset.from_tensor_slices(labels)
test_data = test_data.map(lambda x: str(x))
test_labels = test_labels.map(lambda x: str(x))
writer_data.write(test_data)
writer_labels.write(test_labels)

In [75]:
loaded_dataset = tf.data.TFRecordDataset(["datasets\Mira\labels.tfrecord"])

In [76]:
for d in loaded_dataset.take(1):
    print(d)

tf.Tensor(b'Tensor("args_0:0", shape=(), dtype=int32)', shape=(), dtype=string)


In [77]:
loaded_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

In [83]:
for d in test_data.take(1):
    print(d.numpy())

b'Tensor("args_0:0", shape=(20,), dtype=float64)'


In [74]:
m = tf.keras.metrics.Recall(class_id=1)
m.update_state([1, 0, 1, 2], [1, 0, 1, 2])
m.result().numpy()

InvalidArgumentError: predictions must be <= 1
Condition x <= y did not hold.
First 3 elements of x:
[1. 0. 1.]
First 1 elements of y:
[1.]

In [82]:
y_pred = tf.nn.softmax(model(test_data[:1]))[0]
y_pred

<tf.Tensor: shape=(6,), dtype=float32, numpy=
array([0.98039216, 0.00272254, 0.00372326, 0.00742231, 0.00221274,
       0.00352691], dtype=float32)>

In [98]:
m = tf.keras.metrics.TruePositives()
m.update_state([1, 2, 3], [1, 2, 3])
m.result().numpy()

InvalidArgumentError: predictions must be <= 1
Condition x <= y did not hold.
First 3 elements of x:
[1. 2. 3.]
First 1 elements of y:
[1.]

In [94]:
predictions[:100], labels[:100]

(<tf.Tensor: shape=(100,), dtype=int64, numpy=
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3,
        0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)>,
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 3,
        0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0,
        0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0]))